In [1]:
import pandas as pd

df = pd.read_csv("fish_all.csv")

In [2]:
df.head()

,Species,Weight,Length1,Length2,Length3,Height,Width
0,Bream,242.0,23.2,25.4,30.0,11.5200,4.0200
1,Bream,290.0,24.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,23.9,26.5,31.1,12.3778,4.6961
3,Bream,363.0,26.3,29.0,33.5,12.7300,4.4555
4,Bream,430.0,26.5,29.0,34.0,12.4440,5.1340


In [3]:
df['Species'].value_counts()

Perch        56
Bream        35
Roach        20
Pike         17
Smelt        14
Parkki       11
Whitefish     6
Name: Species, dtype: int64

In [4]:
df.loc[df['Species']=='Perch','Species'] = 0
df.loc[df['Species']=='Bream','Species'] = 1
df.loc[df['Species']=='Roach','Species'] = 2
df.loc[df['Species']=='Pike','Species'] = 3
df.loc[df['Species']=='Smelt','Species'] = 4
df.loc[df['Species']=='Parkki','Species'] = 5
df.loc[df['Species']=='Whitefish','Species'] = 6
df['Species'] = df['Species'].astype(int)

In [5]:
feature = df[df.keys().drop('Species')]
outcome = df['Species']

In [6]:
from sklearn.model_selection import train_test_split

train_feature, test_featue, train_target, test_target = train_test_split(feature,outcome,random_state=42, stratify=outcome)
train_feature, val_featue, train_target, val_target = train_test_split(train_feature,train_target,random_state=42, stratify=train_target)

In [7]:
from lightgbm import LGBMClassifier

clf = LGBMClassifier()
clf.fit(train_feature, train_target)

LGBMClassifier()

In [8]:
y_pred = clf.predict(test_featue)

In [9]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

print("Accuracy :",accuracy_score(test_target, y_pred))
print(confusion_matrix(test_target, y_pred,labels=[6,5,4,3,2,1,0]))

Accuracy : 0.7
[[ 0  0  0  0  0  0  1]
 [ 0  1  0  0  2  0  0]
 [ 0  0  4  0  0  0  0]
 [ 0  0  0  3  0  0  1]
 [ 0  0  0  0  2  0  3]
 [ 2  0  0  0  0  7  0]
 [ 1  0  0  0  2  0 11]]


In [10]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state=42, n_jobs=-1, k_neighbors=3)
train_feature_oversampling, train_target_oversampling = sm.fit_resample(train_feature, train_target)

In [11]:
from lightgbm import LGBMClassifier

clf2 = LGBMClassifier()
clf2.fit(train_feature_oversampling, train_target_oversampling)

LGBMClassifier()

In [12]:
y_pred = clf2.predict(test_featue)

In [13]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

print("Accuracy :",accuracy_score(test_target, y_pred))
print(confusion_matrix(test_target, y_pred,labels=[6,5,4,3,2,1,0]))

Accuracy : 0.625
[[1 0 0 0 0 0 0]
 [0 1 0 0 2 0 0]
 [0 0 4 0 0 0 0]
 [0 0 0 3 0 0 1]
 [1 0 0 0 1 0 3]
 [3 0 0 0 0 6 0]
 [2 0 0 0 3 0 9]]


In [14]:
from lightgbm import LGBMClassifier
from lightgbm.callback import early_stopping

clf3 = LGBMClassifier()
early_stop = early_stopping(stopping_rounds=30,first_metric_only=True)
clf3.fit(train_feature, train_target, eval_set=[(val_featue,val_target)], eval_metric='multi_error',callbacks=[early_stop])

Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[87]	valid_0's multi_error: 0.2	valid_0's multi_logloss: 1.29244
Evaluated only: multi_error


LGBMClassifier()

In [15]:
y_pred = clf3.predict(test_featue)

In [16]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

print("Accuracy :",accuracy_score(test_target, y_pred))
print(confusion_matrix(test_target, y_pred,labels=[6,5,4,3,2,1,0]))

Accuracy : 0.7
[[ 0  0  0  0  0  0  1]
 [ 0  1  0  0  2  0  0]
 [ 0  0  4  0  0  0  0]
 [ 0  0  0  3  0  0  1]
 [ 0  0  0  0  2  0  3]
 [ 1  0  0  0  0  8  0]
 [ 2  0  0  0  2  0 10]]
